In [2]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '1.2.1'


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
    return A, B, L

In [3]:
A, B, L = получить_матрицы_варианта(8)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)

In [4]:
def матрица_управляемости(A, B, L):
    матрица = sympy.eye(3)
    матрица[:, 0] = B 
    матрица[:, 1] = A*B
    матрица[:, 2] = A**2*B
    return матрица

In [5]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[-2, 20,  -72],
[ 0, 14, -100],
[-4,  8, -108]])

In [6]:
def определитель_матрицы_управляемости(S):
    определитель = S.det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [7]:
print(определитель_матрицы_управляемости(S))

5392


In [8]:
def коэффициенты_характеристического_полином_разомкнутой_системы(A):
    l = sympy.symbols('lambda')
    I = sympy.eye(3)
    detChPolyOpen = (l*I - A).det()
    c0 = detChPolyOpen.coeff(l, 0)
    c1 = detChPolyOpen.coeff(l, 1)
    c2 = detChPolyOpen.coeff(l, 2)
    ChPolyOpen = sympy.Matrix([
        c0,
        c1,
        c2
        ])
    return ChPolyOpen

In [9]:
коэффициенты_характеристического_полином_разомкнутой_системы(A)

Matrix([
[-68],
[-14],
[  6]])

In [10]:
def коэффициенты_характеристического_полином_замкнутой_системы(L):
    c0 = L[0]*L[1]*L[2]
    c1 = L[0]*L[1] + L[1]*L[2] + L[0]*L[2]
    c2 = L[0]+L[1]+L[2]

    ChPolyOpen = sympy.Matrix([
        -c0,
        c1,
        -c2
        ])
    return ChPolyOpen

In [11]:
коэффициенты_характеристического_полином_замкнутой_системы(L)

Matrix([
[-90],
[-33],
[  2]])

In [12]:
def вектор_управления(A, B, L):
    s = sympy.symbols('s')
    I = sympy.eye(3)
    матрица = (s*I - A)
    g = матрица.adjugate()*B
    F = матрица.det()
    k0, k1, k2 = коэффициенты_характеристического_полином_замкнутой_системы(L)
    H = s**3 + s**2*k2 + s*k1 + k0

    p1 = sympy.symbols('p_1')
    p2 = sympy.symbols('p_2')
    p3 = sympy.symbols('p_3')

    p = sympy.Matrix([
        p1,
        p2,
        p3
    ]).T

    solved = sympy.solvers.solvers.solve_undetermined_coeffs((p*g)[0] - (H-F), [p1, p2, p3], s)

    res = [float(i) for i in solved.values()]
    res = sympy.Matrix(res)

    return res

In [13]:
res = вектор_управления(A, B, L)
res

Matrix([
[-0.222551928783383],
[0.0400593471810089],
[  1.11127596439169]])

In [14]:
toCopyString(res)

'matrix([-0.222551928783383],[0.0400593471810089],[1.11127596439169])'

In [15]:
for var_number in range(1, 31):
    A, B, L = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)

    S                        = матрица_управляемости(A, B, L)
    определитель             = определитель_матрицы_управляемости(S)
    коэффициенты_разомкнутой = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    коэффициенты_замкнутой   = коэффициенты_характеристического_полином_замкнутой_системы(L)
    вектор_управления_       = вектор_управления(A, B, L)
    
    with open(f'./results/{var_number}.txt', 'w') as file:
        file.write(toCopyString(S) + '\n')
        file.write(str(определитель) + '\n')
        file.write(toCopyString(коэффициенты_разомкнутой) + '\n')
        file.write(toCopyString(коэффициенты_замкнутой) + '\n')
        file.write(toCopyString(вектор_управления_) + '\n')
        